# CLI Tests and Text Extractor from PDF Files
## Provenance
* Created new 2025-05-17 at Bignor!

In [ ]:
from pathlib import Path 
import pandas as pd
import re
import numpy as np
from functools import partial
from pprint import pprint as pp
from random import sample

import pymupdf # fritz  # PyMuPDF
import Levenshtein

%load_ext autotime
%load_ext autoreload

In [ ]:
%autoreload 1
%aimport archivum.mendeley_port
%aimport archivum.library
%aimport archivum.utilities
%aimport archivum.document
%aimport archivum.crossref
%aimport archivum.gui
%aimport archivum.parser
%aimport archivum.reference
%aimport archivum.cli
%aimport archivum.utilities
%aimport archivum.querex

%aimport scratch 
%aimport greater_tables.greater_tables 
%aimport greater_tables.test_tables 
    

import archivum.mendeley_port as arcm
import archivum.library as arcl
import archivum.utilities as arcu
import archivum.document as arcd
import archivum.crossref as arcc
import archivum.gui as arcg
import archivum.parser as arcp
import archivum.reference as arcr
import archivum.cli as cli 

import greater_tables.test_tables as tt
from greater_tables.greater_tables import GT

from archivum.utilities import fGT
import scratch as sc 
%aimport

In [ ]:
lib = arcl.Library('uber-library')
lib.get_library_path_list()

In [ ]:
lib.text_dir_full_name

In [ ]:
str(lib.text_dir_path)

In [ ]:
cli

In [ ]:
cli.entry.commands

In [ ]:
import subprocess
from rich.console import Console
from rich.text import Text
import json
console = Console()

In [ ]:
cmd = ['rg', '--json', '--stats', '-C', '1', 'elicitability', '-g', "*.md", '\\temp\\pdf-full-text']

In [ ]:
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, 
                   encoding='utf-8')

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['get-distinct-values', ], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['rg', r"Hahn\-?Banach theorem"], standalone_mode=False)

In [ ]:
cli.entry(args=["list-libraries"], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)

In [ ]:
cli.entry(args=["list-libraries", "-d"], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)

In [ ]:
cli.entry(args=['rg', 'distortion risk measures'], standalone_mode=False)

In [ ]:
cli.entry(args=['get-library-stats'], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['get-distinct-values', ], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['get-distinct-values', '-f author'], standalone_mode=False)

In [ ]:
lib.list()

In [ ]:
cli.entry(args=['merge-library', 'othername'], standalone_mode=False)

In [ ]:
ans = cli.make_query_completer_static(lib.ref_df)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['query-library'], standalone_mode=False)

In [ ]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['new', '-m'], standalone_mode=False)

# NEW AND IMPORT

In [ ]:
df = cli.LibraryContext.last_new
df.head()

In [ ]:
d = cli.LibraryContext.last_new.iloc[0,0]
d

In [ ]:
txt = d.extract_text()

In [ ]:
d._text_dir_path

In [ ]:
arcu.info(d)

In [ ]:
cli.LibraryContext.last_new.meta_crossref

In [ ]:
r = arcc.search(cli.LibraryContext.last_new.meta_crossref.iloc[14])

In [ ]:
arcl.BASE_DIR

In [ ]:
rans = []
for rf in r:
    ref = arcr.Reference.from_crossref(rf, lib)
    rans.append(ref)
    print(ref.title, ref.author, ref.journal, ref.year)

In [ ]:
for rf in r: print(rf)

In [ ]:
pd.concat([pd.Series(ref.to_dict(), name='value').to_frame() for ref in rans], axis=1).T[['author', 'title', 'year', 'journal']]

In [ ]:
cli.LibraryContext.last_new#.path[0]

In [ ]:
# cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['close-library', ], standalone_mode=False)
cli.entry(args=['get-library-stats'], standalone_mode=False)

In [ ]:
cli.entry(args=["list-libraries", '-d'], standalone_mode=False)

In [ ]:
result = lib.database.querex('top 5 recent verbose')
print(fGT(result))

In [ ]:
f = fGT(lib.querex('top 10 where year == 2024 !/Wang, R/ sort title'))
print(f)

In [ ]:
f = fGT(lib.database.querex('top 10 recent !/Wang, R/'))
print(f)

# GUI

In [ ]:
def edit_book_info():
    import tkinter as tk
    from tkinter import messagebox

    def on_done():
        nonlocal result
        result = {
            "Title": title_var.get(),
            "Author": author_var.get(),
            "Year": year_var.get()
        }
        root.destroy()

    def on_cancel():
        root.destroy()

    result = None
    root = tk.Tk()
    root.title("Edit Book")

    tk.Label(root, text="Title").grid(row=0, column=0, sticky="e")
    tk.Label(root, text="Author").grid(row=1, column=0, sticky="e")
    tk.Label(root, text="Year").grid(row=2, column=0, sticky="e")

    title_var = tk.StringVar(value="The Great Gatsby")
    author_var = tk.StringVar(value="F. Scott Fitzgerald")
    year_var = tk.StringVar(value="1925")

    tk.Entry(root, textvariable=title_var).grid(row=0, column=1)
    tk.Entry(root, textvariable=author_var).grid(row=1, column=1)
    tk.Entry(root, textvariable=year_var).grid(row=2, column=1)

    tk.Button(root, text="Cancel", command=on_cancel).grid(row=3, column=0, pady=10)
    tk.Button(root, text="Done", command=on_done).grid(row=3, column=1, pady=10)

    root.mainloop()
    return result


In [ ]:
import tkinter as tk

def edit_fields(data, mandatory=(), title="Edit Info"):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    root = tk.Tk()
    root.title(title)

    vars_ = {}
    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=2)
        var = tk.StringVar(value=value)
        vars_[key] = var
        tk.Entry(root, textvariable=var).grid(row=row, column=1, padx=5, pady=2)

    row += 1
    tk.Button(root, text="Cancel", width=10, command=on_cancel).grid(row=row, column=0, pady=10)
    tk.Button(root, text="Done", width=10, command=on_done).grid(row=row, column=1, pady=10)

    root.mainloop()
    return result


In [ ]:
bi = {'title': "Once upon a time in America", 
     'author': 'LeLilo, Don.', 
     'year': 2008}
mand = ['title', 'publisher']

In [ ]:
edit_fields(bi, mand)

In [ ]:
edit_book_info()

In [ ]:
import tkinter as tk
from tkinter import ttk

def edit_fields(data, mandatory=(), title="Edit Info", known_values=None):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")

    known_values = known_values or {}
    vars_ = {}

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    def setup_combobox(box, values):
        def on_key(event):
            val = box.get().lower()
            matches = [v for v in values if val in v.lower()]
            if matches:
                box['values'] = matches
            else:
                box['values'] = values
        box.bind('<KeyRelease>', on_key)
        
    root = tk.Tk()
    root.title(title)
    root.geometry("700x400")  # wider window

    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar(value=value)
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key])
            box.config(width=80)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=80).grid(row=row, column=1, padx=5, pady=4)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="right", padx=10)

    root.mainloop()
    return result


In [ ]:
import tkinter as tk
from tkinter import ttk
from rapidfuzz import process

def edit_fields(data, mandatory=(), title="Edit Info", known_values=None):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")
    
    FONT = ("Segoe UI", 10)

    known_values = known_values or {}
    vars_ = {}

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    def on_next():

    def on_previous()
    
    def setup_fuzzy_combobox(box, values):
        def on_key(event):
            current = box.get()
            if current:
                match, score, _ = process.extractOne(current, values, score_cutoff=40)
                if match:
                    box.set(match)
        box.bind('<FocusOut>', on_key)
        box.bind('<Tab>', on_key)

    root = tk.Tk()
    root.title(title)
    root.geometry("800x500")

    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar(value=value)
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key], width=80, font=FONT)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_fuzzy_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=82, font=FONT).grid(row=row, column=1, padx=5, pady=4)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="right", padx=10)
    tk.Button(btn_frame, text="Previous", width=12, command=on_previous).pack(side="right", padx=10)
    tk.Button(btn_frame, text="Next", width=12, command=on_next).pack(side="right", padx=10)

    root.mainloop()
    return result


In [ ]:
import tkinter as tk
from tkinter import ttk
from rapidfuzz import process
import pandas as pd

def edit_fields_from_df(df, i, mandatory=(), title="Edit Row", known_values=None):
    result = None
    rowcount = len(df)
    fields = df.iloc[i].to_dict()
    for key in mandatory:
        fields.setdefault(key, "")

    known_values = known_values or {}
    vars_ = {}

    def load_row(idx):
        for k in fields:
            val = str(df.iloc[idx].get(k, ""))
            vars_[k].set(val)

    def save_row():
        return {k: vars_[k].get() for k in fields}

    def on_done():
        nonlocal result
        result = (save_row(), current[0])
        root.destroy()

    def on_cancel():
        root.destroy()

    def on_prev():
        if current[0] > 0:
            current[0] -= 1
            load_row(current[0])

    def on_next():
        if current[0] < rowcount - 1:
            current[0] += 1
            load_row(current[0])

    def setup_fuzzy_combobox(box, values):
        def on_key(event):
            current = box.get()
            if current:
                match, score, _ = process.extractOne(current, values, score_cutoff=70)
                if match and match != current:
                    box.set(match)
                    box.icursor(tk.END)
        box.bind('<KeyRelease>', on_key)

    current = [i]  # mutable index holder
    root = tk.Tk()
    root.title(title)
    root.geometry("800x500")

    for row, key in enumerate(fields):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar()
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key], width=80)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_fuzzy_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=80).grid(row=row, column=1, padx=5, pady=4)

    load_row(i)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Previous", width=12, command=on_prev).pack(side="left", padx=5)
    tk.Button(btn_frame, text="Next", width=12, command=on_next).pack(side="left", padx=5)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=20)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="left", padx=5)

    root.mainloop()
    return result


In [ ]:
mandatory = ['type', 'author', 'title', 'year', 'journal', 'publisher']
defaults = {
    k: lib.distinct(k) for k in mandatory
}

In [ ]:
# def edit_fields_from_df(df, i, mandatory=(), title="Edit Row", known_values=None):
updated = edit_fields_from_df(lib.ref_df.sample(20).reset_index(drop=True)[mandatory], 0, mandatory, title="Edit Book Info", known_values=defaults)
updated